In [0]:
%fs ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/name_basics.tsv,name_basics.tsv,878170765,1742916710000
dbfs:/FileStore/tables/title_basics.tsv,title_basics.tsv,996901919,1742916733000
dbfs:/FileStore/tables/title_ratings.tsv,title_ratings.tsv,26949937,1742916723000


Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [0]:
filePath = "dbfs:/FileStore/tables/name_basics.tsv"
namesDf = spark.read.format("csv") \
              .option("header","true") \
              .option("delimiter", "\t") \
              .option("inferSchema","true") \
              .load(filePath)

In [0]:
display(namesDf.head(20))

nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0027125,tt0031983"
nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"
nm0000006,Ingrid Bergman,1915,1982,"actress,producer,soundtrack","tt0034583,tt0038109,tt0036855,tt0038787"
nm0000007,Humphrey Bogart,1899,1957,"actor,producer,miscellaneous","tt0034583,tt0043265,tt0037382,tt0033870"
nm0000008,Marlon Brando,1924,2004,"actor,director,writer","tt0078788,tt0068646,tt0047296,tt0044081"
nm0000009,Richard Burton,1925,1984,"actor,producer,director","tt0061184,tt0087803,tt0059749,tt0057877"
nm0000010,James Cagney,1899,1986,"actor,director,producer","tt0029870,tt0031867,tt0042041,tt0034236"


### Tabela name_basics, pobrana z IMDb, nie ma tych niektórych kolumn, więc trochę pozmieniam te podpunkty

- Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
- Dodaj kolumnę w której wyliczysz główną porfesję
- Odpowiedz na pytanie jakie jest najpopularniesze imię?
- Dodaj kolumnę i policz wiek aktorów
- Usuń kolumny (nconst)
- Zmień nazwy kolumn - dodaj kapitalizaję i dodaj spacje
- Posortuj dataframe po imieniu rosnąco

In [0]:
from pyspark.sql.functions import unix_timestamp, current_date, split, element_at, col, desc, asc, when, lit, year

NamesModifiedDf = namesDf
# epoch
NamesModifiedDf = NamesModifiedDf.withColumn("timeOfExecution", unix_timestamp(current_date()))
# main profession
NamesModifiedDf = NamesModifiedDf.withColumn("mainProfession", 
                                            element_at(split(col("primaryProfession"), ","), 1))
# most popular name:
mostPopularNameDf = NamesModifiedDf.withColumn("firstName", 
                                             element_at(split(col("primaryName"), " "), 1)) \
                                   .select("firstName") \
                                   .groupBy("firstName") \
                                   .count() \
                                   .orderBy(desc("count"))

mostPopularName = mostPopularNameDf.first()

# Actors age
current_year = year(current_date())

# Calculate age
NamesModifiedDf = NamesModifiedDf.withColumn(
    "age",
    when(
        (col("deathYear") == "\\N") | (col("deathYear").isNull()),
        current_year - col("birthYear").cast("int")  # Still alive
    ).otherwise(
        col("deathYear").cast("int") - col("birthYear").cast("int")  # Deceased
    )
)

# Dropping nconst 
NamesModifiedDf = NamesModifiedDf.drop("nconst")

# Column capitalization and added spaces
def format_column_name(col_name):
    spaced_name = ''.join([' ' + c if c.isupper() else c for c in col_name])
    spaced_name = spaced_name.strip()
    return ' '.join(word.capitalize() for word in spaced_name.split(' '))

old_columns = NamesModifiedDf.columns
new_columns = [format_column_name(col) for col in old_columns]
for old_col, new_col in zip(old_columns, new_columns):
    NamesModifiedDf = NamesModifiedDf.withColumnRenamed(old_col, new_col)

# Sort by name ascending
NamesModifiedDf = NamesModifiedDf.orderBy(asc("Primary Name"))

display(NamesModifiedDf.head(20))
print(f"The most popular first name is '{mostPopularName['firstName']}': {mostPopularName['count']}")

Primary Name,Birth Year,Death Year,Primary Profession,Known For Titles,Time Of Execution,Main Profession,Age
!'aru Ikhuisi Piet Berendse,\N,\N,\N,tt4245110,1742860800,\N,null
!Gubi Tietie,\N,\N,actor,tt2364626,1742860800,actor,null
!Mystery Girl!,\N,\N,actress,tt0136517,1742860800,actress,null
!Nqate Xqamxebe,\N,\N,\N,tt0262452,1742860800,\N,null
!llmind,\N,\N,\N,\N,1742860800,\N,null
"""Cowboy"" Eddie Long",\N,\N,\N,\N,1742860800,\N,null
"""GALA"" Halyna Koksharova",\N,\N,\N,\N,1742860800,\N,null
"""Jungle Joe"" Fortunato",\N,\N,\N,\N,1742860800,\N,null
"""Laughing""Larry Berger",\N,\N,\N,\N,1742860800,\N,null
"""Luna"" Ashlee Searles",\N,\N,\N,\N,1742860800,\N,null


The most popular first name is 'David': 126173


Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [0]:
filePath = "dbfs:/FileStore/tables/title_basics.tsv"
titlesDf = spark.read.format("csv") \
              .option("header","true") \
              .option("delimiter", "\t") \
              .option("inferSchema","true") \
              .load(filePath)

display(titlesDf.head(20))

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short
tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


### Tutaj to samo, więc lekko zmodyfikuje
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże długość filmu w godzinach
* Usuń wiersze z dataframe gdzie wartości są null


In [0]:
TitlesModifiedDf = titlesDf
# remove nulls
TitlesModifiedDf = TitlesModifiedDf.na.drop()

# epoch
TitlesModifiedDf = TitlesModifiedDf.withColumn("timeOfExecution", unix_timestamp(current_date()))

# time from publication
current_year = year(current_date())

# Calculate age
TitlesModifiedDf = TitlesModifiedDf.withColumn(
    "age",
    current_year - col("startYear")
)

from pyspark.sql.functions import round

# Calculate time in hours
TitlesModifiedDf = TitlesModifiedDf.withColumn(
    "runtimeHours",
    round(col("runtimeMinutes").cast("float") / 60, 2)
)

# remove nulls
TitlesModifiedDf = TitlesModifiedDf.na.drop()

display(TitlesModifiedDf.head(20))

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,timeOfExecution,age,runtimeHours
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1742860800,131.0,0.02
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",1742860800,133.0,0.08
tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance",1742860800,133.0,0.08
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",1742860800,133.0,0.2
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short,1742860800,132.0,0.02
tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short,1742860800,131.0,0.02
tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport",1742860800,131.0,0.02
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short",1742860800,131.0,0.02
tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,1742860800,131.0,0.75
tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short",1742860800,130.0,0.02


ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long` 

In [0]:
filePath = "dbfs:/FileStore/tables/title_ratings.tsv"
ratingsDf = spark.read.format("csv") \
              .option("header","true") \
              .option("delimiter", "\t") \
              .option("inferSchema","true") \
              .load(filePath)

display(ratingsDf.head(20))

tconst,averageRating,numVotes
tt0000001,5.7,2142
tt0000002,5.5,290
tt0000003,6.4,2178
tt0000004,5.3,186
tt0000005,6.2,2912
tt0000006,5.0,209
tt0000007,5.3,904
tt0000008,5.4,2289
tt0000009,5.4,222
tt0000010,6.8,7909


In [0]:
joinedDf = TitlesModifiedDf.join(ratingsDf, on="tconst")
display(joinedDf.head(30))
RatingsModifiedDf = joinedDf

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,timeOfExecution,age,runtimeHours,averageRating,numVotes
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",1742860800,133.0,0.08,5.5,290
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",1742860800,133.0,0.2,5.3,186
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short",1742860800,131.0,0.02,5.4,2289
tt0000015,short,Around a Cabin,Autour d'une cabine,0,1894,\N,2,"Animation,Comedy,Short",1742860800,131.0,0.03,6.1,1266
tt0000017,short,Italienischer Bauerntanz,Italienischer Bauerntanz,0,1895,\N,1,"Documentary,Short",1742860800,130.0,0.02,4.6,378
tt0000108,short,Rip Leaving Sleepy Hollow,Rip Leaving Sleepy Hollow,0,1896,\N,1,"Drama,Short",1742860800,129.0,0.02,4.4,587
tt0000169,short,Slagsmål i gamla Stockholm,Slagsmål i gamla Stockholm,0,1897,\N,1,Short,1742860800,128.0,0.02,4.0,61
tt0000170,short,The Twins' Tea Party,The Twins' Tea Party,0,1896,\N,1,"Comedy,Short",1742860800,129.0,0.02,5.2,295
tt0000172,short,The X-Ray Fiend,The X-Ray Fiend,0,1897,\N,1,"Comedy,Horror,Short",1742860800,128.0,0.02,6.0,1141
tt0000174,short,Výstavní párkar a lepic plakátù,Výstavní párkar a lepic plakátù,0,1898,\N,1,"Comedy,Short",1742860800,127.0,0.02,4.5,147


### Tutaj to samo, więc kolejne modyfikacje

* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Jaki film ma najlepsze oceny, oraz jaka jest średnia ocena filmów
* Dla jednej z kolumn zmień typ danych do `long` 

In [0]:
from pyspark.sql.functions import unix_timestamp, current_date, desc, avg

# Add timestamp and drop nulls
RatingsModifiedDf = RatingsModifiedDf.withColumn("timeOfExecution", unix_timestamp(current_date()))
RatingsModifiedDf = RatingsModifiedDf.na.drop()

# Best Rated
best_rated_row = RatingsModifiedDf.orderBy(desc("averageRating")).first()
best_film_title = best_rated_row["primaryTitle"]
best_film_original = best_rated_row["originalTitle"]
best_film_rating = best_rated_row["averageRating"]

# Average
avg_rating = RatingsModifiedDf.select(avg("averageRating")).first()[0]

# Cast to long
RatingsModifiedDf = RatingsModifiedDf.withColumn("startYear", col("startYear").cast("long"))

# Print results
print(f"Best Rated Film: {best_film_title} ({best_film_original}), Rating: {best_film_rating}")
print(f"Average Rating: {avg_rating}")

# Display dataframe
display(RatingsModifiedDf.head(20))
RatingsModifiedDf.printSchema()

Best Rated Film: The Mark (The Mark), Rating: 10.0
Average Rating: 6.855942238493382


tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,timeOfExecution,age,runtimeHours,averageRating,numVotes
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",1742860800,133.0,0.08,5.5,290
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",1742860800,133.0,0.2,5.3,186
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short",1742860800,131.0,0.02,5.4,2289
tt0000015,short,Around a Cabin,Autour d'une cabine,0,1894,\N,2,"Animation,Comedy,Short",1742860800,131.0,0.03,6.1,1266
tt0000017,short,Italienischer Bauerntanz,Italienischer Bauerntanz,0,1895,\N,1,"Documentary,Short",1742860800,130.0,0.02,4.6,378
tt0000108,short,Rip Leaving Sleepy Hollow,Rip Leaving Sleepy Hollow,0,1896,\N,1,"Drama,Short",1742860800,129.0,0.02,4.4,587
tt0000169,short,Slagsmål i gamla Stockholm,Slagsmål i gamla Stockholm,0,1897,\N,1,Short,1742860800,128.0,0.02,4.0,61
tt0000170,short,The Twins' Tea Party,The Twins' Tea Party,0,1896,\N,1,"Comedy,Short",1742860800,129.0,0.02,5.2,295
tt0000172,short,The X-Ray Fiend,The X-Ray Fiend,0,1897,\N,1,"Comedy,Horror,Short",1742860800,128.0,0.02,6.0,1141
tt0000174,short,Výstavní párkar a lepic plakátù,Výstavní párkar a lepic plakátù,0,1898,\N,1,"Comedy,Short",1742860800,127.0,0.02,4.5,147


root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: long (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- timeOfExecution: long (nullable = true)
 |-- age: double (nullable = true)
 |-- runtimeHours: double (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)



# 2 Spark UI

## Elementy Spark UI

### Jobs
Jobs pokazują wysokopoziomowe zadania Sparka. Można tu zobaczyć poszczególne wywołania akcji (np. collect(), count()), ich status (sukces/porażka), czas wykonania oraz szczegóły każdego zadania. Pozwala to analizować wydajność i identyfikować wąskie gardła w aplikacji.

### Stages
Stages pokazują jak jobs są dzielone na mniejsze grupy zadań. Wyświetla informacje o czasie wykonania, liczbie zadań, odczytanych/zapisanych danych i shuffle operations. Dostępne są szczegółowe metryki dotyczące wykonywania poszczególnych etapów.

### Storage
Zakładka Storage wyświetla informacje o danych przechowywanych w pamięci (cached RDDs i DataFrames). Pokazuje poziom wykorzystania pamięci, rodzaj przechowywania (MEMORY_ONLY, MEMORY_AND_DISK) oraz stopień partycjonowania danych.

### Environment
Environment zawiera informacje o konfiguracji środowiska Spark, w tym wersję Spark, parametry JVM, zmienne środowiskowe i właściwości Sparka. Jest to przydatne do debugowania problemów związanych z konfiguracją.

### Executors
Executors pokazuje informacje o węzłach wykonawczych Sparka, w tym wykorzystanie zasobów (CPU, pamięć), ilość przetworzonych danych, liczbę aktywnych/zakończonych zadań oraz metryki GC. Pomaga monitorować równomierność obciążenia klastra.

### SQL/DataFrame
Ta zakładka pokazuje wykonanie zapytań SQL i operacji na DataFrame'ach. Zawiera plany fizyczne i logiczne, statystyki wykonania oraz wizualizacje grafów DAG dla zapytań. Pozwala analizować złożone operacje i ich optymalizację.

### JDBC/ODBC Server
Wyświetla informacje o połączeniach JDBC/ODBC do Spark Thrift Server, w tym aktywne sesje, wykonywane zapytania i statystyki połączeń. Jest to przydatne przy korzystaniu ze Sparka jako źródła danych dla narzędzi BI.

### Structured Streaming
Ta sekcja zawiera informacje o aplikacjach Structured Streaming, w tym przepustowość, opóźnienia, aktywne zapytania strumieniowe oraz statystyki dotyczące źródeł i ujść danych. Pomaga monitorować przetwarzanie strumieniowe w czasie rzeczywistym.